https://documenter.getpostman.com/view/13485071/2s93sgXWUY?utm_campaign=API&utm_source=gs&utm_medium=email&utm_content=sign-up#ea172e5b-8467-4c95-b5e9-83288ee94e24

In [25]:
import sys

# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install geopy

You are using pip version 19.0.3, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [26]:
import pandas as pd
import requests
import configparser
# from geopy.geocoders import Nominatim

In [27]:
# def get_lat_lon_from_zip(zip_code):
#     geolocator = Nominatim(user_agent="geoapiExercises")
#     location = geolocator.geocode(zip_code)
#     if location:
#         return (location.latitude, location.longitude)
#     else:
#         return None

In [31]:
def get_lat_lon_from_zip(zip_code, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={zip_code}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            location = data['results'][0]['geometry']['location']
            return (location['lat'], location['lng'])
        else:
            return None
    else:
        return None

In [30]:
# Get Great Schools API Credentials
config = configparser.ConfigParser()
config.read('../SECRETS.ini')
GS_API_KEY = config['great_schools_api']['key']
GG_API_KEY = config['google_geocode_api']['key']

In [33]:
zip_code = 95121

lat, lon = get_lat_lon_from_zip(zip_code, GG_API_KEY)
lat, lon

(37.302108, -121.8050125)

In [23]:
url = f'https://gs-api.greatschools.org/schools?zip={zip_code}'
payload={}
headers = {
  'Accept': 'application/json',
  'Content': 'application/json',
  'X-API-Key': GS_API_KEY
}

response = requests.request("GET", url, headers=headers, data=payload)

response.json()

{'schools': [{'universal-id': '0610273',
   'nces-id': 'A9500584',
   'state-id': '43694357008956',
   'name': 'Scholars Academy',
   'school-summary': 'Scholars Academy, a private school located in San Jose, CA, serves grades K-3 in the .',
   'type': 'private',
   'level-codes': 'e',
   'level': 'KG,1,2,3',
   'street': '3703 Silver Creek Road',
   'city': 'San Jose',
   'state': 'CA',
   'fipscounty': 6085,
   'zip': '95121',
   'phone': '(408) 238-2500',
   'fax': None,
   'county': 'Santa Clara County',
   'lat': 37.301971,
   'lon': -121.806412,
   'district-name': None,
   'district-id': 0,
   'web-site': 'http://scholarsacademyschool.com/',
   'overview-url': 'https://www.greatschools.org/california/san-jose/10273-Scholars-Academy/',
   'rating': None,
   'year': None,
   'rating-description': 'The GreatSchools Rating helps parents compare schools within a state based on a variety of school quality indicators and provides a helpful picture of how effectively each school serves 

In [34]:
url = f'https://gs-api.greatschools.org/nearby-schools?lat={lat}&lon={lon}&limit=50'
payload={}
headers = {
  'Accept': 'application/json',
  'Content': 'application/json',
  'X-API-Key': API_KEY
}

response = requests.request("GET", url, headers=headers, data=payload)

response.json()

{'schools': [{'universal-id': '0633620',
   'nces-id': 'A1900720',
   'state-id': None,
   'name': 'Sofos Preparatory Academy',
   'school-summary': 'Sofos Preparatory Academy, a private school located in San Jose, CA, serves grades 8-11 in the .',
   'type': 'private',
   'level-codes': 'm,h',
   'level': '8,9,10,11',
   'street': '1009 E CAPITOL EXPY # 610',
   'city': 'San Jose',
   'state': 'CA',
   'fipscounty': 6085,
   'zip': '95121',
   'phone': '(408) 667-9277',
   'fax': None,
   'county': 'SANTA CLARA',
   'lat': 37.302109,
   'lon': -121.805016,
   'district-name': None,
   'district-id': 0,
   'web-site': None,
   'overview-url': 'https://www.greatschools.org/california/san-jose/33620-Sofos-Preparatory-Academy/',
   'rating': None,
   'year': None,
   'rating-description': 'The GreatSchools Rating helps parents compare schools within a state based on a variety of school quality indicators and provides a helpful picture of how effectively each school serves all of its stude